In [ ]:
from flask import *
import sqlite3 as sql

app=Flask(__name__)
app.secret_key='login'

@app.route("/")
def login():
    print(session.get("username"))
    if(session.get("username") != None ):
        print("User Already Logged In!")
        return redirect(url_for("dashboard"))
    
    return render_template("login.html")


@app.route("/fail/<msg>", methods=["GET","POST"])
def fail(msg):
    return render_template("result.html",message=msg)



@app.route("/registerpage" ,methods=["GET","POST"])
def registerpage():
    return render_template("registerpage.html")

@app.route("/dashboard",methods=["GET","post"])
def dashboard():
    print(session.get("username"))
    cnt = session.get("count")
    
    if cnt == None:
        cnt = 1
    else:
        cnt = int(cnt) + 1
    
    print(cnt)
    
    if(session.get("username") == None):
        print("No User Logged In")
        #return redirect(url_for("login",message=msg))
        return render_template("login.html")
    
    session["count"]=cnt
    with sql.connect("database.db") as con:
        cur = con.cursor() 
        cur.execute('SELECT name FROM studinfo WHERE email = ?', [session['username']])
        user = str(cur.fetchone())
    
        username= session["username"]
        resp = (render_template("dashboard.html",loginuser=user,count=cnt))
        return resp

@app.route("/profilepage",methods=["GET","post"])
def profilepage():
    if 'loggedin' in session:
        print ("a - "+session["username"])
        username= session["username"]
        with sql.connect("database.db") as con:
            cur = con.cursor() 
            cur.execute('SELECT * FROM studinfo WHERE email = ?', [session['username']])
            user = cur.fetchone()
            print(user)
            return render_template('profilepage.html', userinfo=user)
     
    return redirect(url_for('login'))

@app.route("/profileupdatepage",methods=["GET","post"])
def profileupdatepage():
        return render_template("profileupdatepage.html")

@app.route("/changepassword",methods=["GET",'POST'])
def changepassword():
    return render_template("changepassword.html")


@app.route("/logout")
def logout():
    session.pop("username")
    return redirect(url_for("login"))



@app.route("/addrec",methods=["GET","POST"])
def addrec():
    if request.method =='POST':
        try:
            nm=request.form['name']
            email=request.form['email']
            mobile=request.form['mobile']
            edu=request.form['edu']
            passw=request.form['pass']
            with sql.connect('database.db') as con:
                cur=con.cursor()
                cur.execute("INSERT INTO studinfo (name,email,mobile,education,password) VALUES (?,?,?,?,?)",(nm,email,mobile,edu,passw))
                con.commit()
                msg="Thank you...You Are Registered Successfully!"
        except:
            con.rollback()
            msg="Error in inserting Details...!"
        
        finally:
            return render_template("login.html",msg=msg)
            conn.close()
            
@app.route('/updaterec',methods = ['POST', 'GET'])
def updaterec(): 
    if 'loggedin' in session:
        username=[session['username']]
        if request.method == 'POST':
            try:
                nm = request.form['name']
                email = request.form['email']
                mobile = request.form['mobile']
                education = request.form['education']
                with sql.connect("database.db") as con:
                    cur = con.cursor()
                    cur.execute("UPDATE studinfo SET name=?, email=?, mobile=?, education=? WHERE email=?",(nm,email,mobile,education,session['username']) )
                    user = cur.fetchone()
                    con.commit()
                    msg = "Record successfully Updated"
            except:
                con.rollback()
                msg = "Error in update operation"
      
            finally:
                  #return render_template("result.html",message=msg)
                  return redirect(url_for("profilepage"))
                  con.close()

        


@app.route("/process", methods=["GET","POST"])
def process():
    if request.method == 'POST':
        tuser=request.form["tuser"]
        tpass=request.form["tpass"]
        with sql.connect("database.db") as con:
                cur = con.cursor()
                cur.execute('SELECT * FROM studinfo WHERE email = ? AND password = ?', (tuser, tpass))
                user = cur.fetchone()
                if user:
                    session['loggedin'] = True
                    session['username'] = tuser
                    return redirect(url_for("dashboard",msg="login successfull"))
                else:
                    #return redirect(url_for("fail",msg="sorry,login invalid !!"))
                    return render_template("login.html",msg="Sorry Invalid Login Details..!")
    
    return render_template('result.html', message=msg)
    
        
@app.route("/changepass",methods=["GET","POST"])
def changepass():
    if 'loggedin' in session:
        if request.method=='POST':
           # try:
                npass=request.form['npass']
                cpass=request.form['cpass']
                if (npass==cpass):
                    with sql.connect('database.db') as con:
                        cur=con.cursor()
                        cur.execute("UPDATE studinfo SET password=? WHERE email=?",(cpass,session['username']))
                        con.commit()
                        msg = "Password  successfully Updated"
                        return render_template("changepassword.html",message=msg)
                        con.close()
                else:
                    #con.rollback()
                    msg = "Error in update operation"
                    return render_template("changepassword.html",message=msg)
                    con.close()
             
        # except:
         #       con.rollback()
         #       msg = "Error in update operation"
        
                
          #  finally:
           #       return render_template("result.html",message=msg)
            #      con.close()
        
                
            

        
            
@app.route('/list')
def list():
    
    con = sql.connect("database.db")
    con.row_factory = sql.Row
   
    cur = con.cursor()
    cur.execute("select * from studinfo")
   
    rows = cur.fetchall();
    return render_template("list.html",rows = rows)
            
            

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Dec/2019 22:57:58] "GET / HTTP/1.1" 200 -


None


127.0.0.1 - - [21/Dec/2019 22:58:24] "GET /list HTTP/1.1" 200 -
